# Previsão de Qualidade do Ar

## Proposta

Listar cada uma das features com o seu significado

---
## Análise Exploratória

Analisar cada uma das features e analisar sua distribuição

In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
# estatisticas das features
df.describe()

NameError: name 'df' is not defined

In [ ]:
# distribuicao das features numericas
sns.histplot(df['feature_name'])
plt.title('chart title')
plt.show()

---
## Pré Processamento

### Valores Duplicados

In [ ]:
# verificação de existência de valores duplicados
print(df.duplicated().sum())
# remoçao de valores duplicados
df.drop_duplicates()

### Valores Faltantes

In [ ]:
# verificação de valores faltantes
print(df.isnull().sum())
# remoção de linhas com valores nulos
df_clean = df.dropna()
# para preencher valores faltantes com a média da coluna
df_filled = df.fillna(df.mean())

### Codificação de Variáveis Categóricas

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Instanciando a classe do OneHotEncoder
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(df[['categoric_feature']]).toarray()
df_encoded = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['categoric_feature']))

# Concatenar os novos dados codificados ao dados
df_encoded = pd.concat([df, df_encoded], axis=1)
df_encoded = df_encoded.drop(['categoric_feature'], axis=1)
df_encoded.head()

---
## Divisão dos Dados em Treino e Teste

In [ ]:
# divisão dos dados
X = df.drop(['pm25'], axis=1)
y = df['pm25'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Normalização

In [ ]:
from sklearn.preprocessing import Normalizer

# Inicializar o Normalizer
normalizer = Normalizer.fit(X)
# Normalização entre 0 e 1
X_train_normalized = normalizer.transform(X_train)
X_test_normalized = normalizer.transform(X_test)

### Padronização

In [ ]:
from sklearn.preprocessing import StandardScaler

# Inicializar o StandardScaler
scaler = StandardScaler()

# Ajustar o scaler apenas na base de treino
scaler.fit(X_train)

# Aplicar a transformação nas bases de treino e teste
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

---
## Definição de Métricas de Avaliação

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
# função para testar modelos de regressão e retornar valores das métricas
def reg_evaluation(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    print(f'MSE: {mse}\nRMSE: {rmse}\nMAPE: {mape}')
    return

---
## Treinamento dos Modelos

### Regressão Linear

In [ ]:
from sklearn.linear_model import LinearRegression

# criando e treinando o modelo
reg_model = LinearRegression()
reg_evaluation(reg_model, x_train, y_train, x_test, y_test)

### KNN Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# criando e treinando o modelo
knn_reg_model = KNeighborsRegressor()
reg_evaluation(knn_reg_model, x_train, y_train, x_test, y_test)

### SVR

In [ ]:
from sklearn.svm import SVR

# criando e treinando o modelo
svr_model = SVR()
reg_evaluation(svr_model, x_train, y_train, x_test, y_test)

### Árvore de Regressão

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# criando e treinando o modelo
tree_reg_model = DecisionTreeRegressor()
reg_evaluation(tree_reg_model, x_train, y_train, x_test, y_test)

### XGBoost

In [ ]:
from xgboost import XGBRegressor

# criando e treinando o modelo
xgb_model = XGBRegressor()
reg_evaluation(xgb_model, x_train, y_train, x_test, y_test)

---
## Otimização do Modelo

### Grid Search com Validação Cruzada

In [ ]:
from sklearn.model_selection import GridSearchCV

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

# Definindo os parâmetros a serem ajustados
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

# Criando o modelo
svr = SVR()

# Ajuste fino com validação cruzada
clf = GridSearchCV(svr, parameters, cv=10)

# Treinando o modelo com otimização
clf.fit(x_train, y_train)